# Parse development notebook

### Notebook purpose
This notebook is development space for python parse.ts replacement and upgrade.
It reads specified google sheets and output actants.json file, which can be imported to inkVisitor RethinkDB.py

The aggreated doc for dev: https://docs.google.com/document/d/1ga6R_9TWAQlXE9XqPE_qZ2S8S8aVW2A7n8SuYpSXnoI/edit#heading=h.q9ntf0ofam2u
The holy schema: https://app.diagrams.net/#G1bKvqEKr6JzPWryVg-vYudQy_4KvuzHS9

### Prerequisites
 * generated json schema for all used objects (run generate-json-schemas.py)

### JSon schemas for the actants
...
...

### The import tables:
 * Texts
 * Manuscripts (must be done alongside T, David thinks) = O of class defined by col. class_id
   * will have legacyId M
   * Prostě je přeparsujeme na O s metaprop "has - class - manuscript codex" (see class_id)
   *
 * Resources
 * Concepts
 * Actions

... and  **VALUES** (virtual dataset made / filled through the import and appended to the entities json at the end)

### Notes
 * DONE: tables contain hyperlinks formulas, Dator was updated to handle them in resulting "label|url" format in the cells
 * input table contain multiples and input_values meant as VALUE entities (~V~something)

### TODOS
#### A
 * make code for creation of entity dataset, which will generate hash id, i.e. I need do NOT make relationt through legacyId
 *
#### B
 * ..
 *
#### C
 * ..
 *


**Goal:**  to have a json file full of territories, which will be added to the mock up actants.json file and inserted to RethinkDb and fully visisble, accessible in inkVisitor.

Steps
 * study the mockup actants.json structure  DONE
 * prepare a code intepreting and operatinalizing David's import instruction in the header_df  DONE
 * parse the table to the territories.json
   * helper class for "territory", which fills default values
   *
 * merge json files
 * import ...

Forseen complexities
 * '#' hash notation, so called *multiples*, in the id fields, C2015#C0156. ...
 * '~' tilda notation
 * SOLVED, urls in cell (formula =HYPERLINK), SOLVED in Dator class

### Input variables

In [339]:
input_tables = ["texts", "manuscripts", "resources", "actions" , "concepts"]

#                   sheet_name,  code, header_in_row
input_sheets = {
    "texts" : ("Texts","13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5), #https://docs.google.com/spreadsheets/d/13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE/edit#gid=2056508047
    "manuscripts" : ("Manuscripts", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "resources" : ("Resources", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "actions" :  ("Statements","1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ", 5), # https://docs.google.com/spreadsheets/d/1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ/edit#gid=0
    "concepts" : ("Concepts","1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4", 5) # https://docs.google.com/spreadsheets/d/1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4/edit#gid=0
}

table_to_entity = {
    "texts" : "ITerritory",
    "manuscripts" : "IObject",
    "resources" : "IResource",
    "actions" :  "IAction",
    "concepts" : "IConcept"
}

root_sheet_url = "https://docs.google.com/spreadsheets/d/"
google_api_dotenv_path = "../env/.env.googleapi"  # contains google api specs for sheet access with Dator
schema_path = '../schemas/' # path for dir with schemas
json_schemas = {}  # holder for schemas, so they can be used for jsonschema validate

In [405]:
import subprocess
subprocess.run("python generate-json-schemas.py", shell=True,capture_output=True)

CompletedProcess(args='python generate-json-schemas.py', returncode=0, stdout=b'typescript-json-schema ../../shared/types/actant.ts IActant --required --noExtraProps --defaultProps --id IActant --out ../schemas/IActant.schema.json\r\ntypescript-json-schema ../../shared/types/concept.ts IConcept --required --noExtraProps --defaultProps --id IConcept --out ../schemas/IConcept.schema.json\r\ntypescript-json-schema ../../shared/types/object.ts IObject --required --noExtraProps --defaultProps --id IObject --out ../schemas/IObject.schema.json\r\ntypescript-json-schema ../../shared/types/entity.ts IEntity --required --noExtraProps --defaultProps --id IEntity --out ../schemas/IEntity.schema.json\r\ntypescript-json-schema ../../shared/types/label.ts ILabel --required --noExtraProps --defaultProps --id ILabel --out ../schemas/ILabel.schema.json\r\ntypescript-json-schema ../../shared/types/statement.ts IStatement --required --noExtraProps --defaultProps --id IStatement --out ../schemas/IStatement

### Libraries

In [341]:
import os, warlock, json
from datetime import datetime
from jsonschema import validate
import dissinetpytools.dator as dator
from dotenv import load_dotenv

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import numpy as np
from copy import deepcopy
from shutil import copyfile

import uuid

def get_uuid_id():
    return str(uuid.uuid4())

def is_valid_uuid(val):
    try:
        uuid.UUID(str(val))
        return True
    except ValueError:
        return False


# type hinting
from collections.abc import Sequence, Callable
from typing import List, Dict, Tuple


### Initialisation

In [342]:
load_dotenv(google_api_dotenv_path) # fills os.environ['GDRIVE_API_CREDENTIALS']
d = dator.Dator(loglevel=10, print_log_online=True, cache=True, project_name="inkvisitor-import") # expects 'GDRIVE_API_CREDENTIALS' in the global system variables (os.environ)
d.google_authenticate()
logger = d.logger

20 2022-03-08 23:13:02 : Google authentification start
20 2022-03-08 23:13:02 : Google authentification end
20 2022-03-08 23:13:02 : Dator initiation succesfull end


In [343]:
# read all schemas inside and warlock them as globally available classes
schema_filenames = os.listdir(schema_path)
json_classes = {}
for schema in schema_filenames:
    name = schema.split(".")[0]
    file_handler = open(schema_path + schema,"r")
    schema_json = json.load(file_handler)
    json_schemas[name] = schema_json
    globals()[name] = warlock.model_factory(schema_json)
    json_classes[name] = globals()[name]
    logger.info("Class " + name + " available.")

logger.info(f"There are {len(json_classes.keys())} json classes available ({' '.join(json_classes.keys())}).")

2022-03-08 23:13:02,134 INFO Class IActant available.
2022-03-08 23:13:02,143 INFO Class IAction available.
2022-03-08 23:13:02,152 INFO Class IConcept available.
2022-03-08 23:13:02,160 INFO Class IEntity available.
2022-03-08 23:13:02,167 INFO Class ILabel available.
2022-03-08 23:13:02,174 INFO Class IProp available.
2022-03-08 23:13:02,182 INFO Class IResource available.
2022-03-08 23:13:02,192 INFO Class IStatement available.
2022-03-08 23:13:02,200 INFO Class ITerritory available.
2022-03-08 23:13:02,209 INFO Class IUser available.
2022-03-08 23:13:02,217 INFO Class IValue available.
2022-03-08 23:13:02,218 INFO There are 11 json classes available (IActant IAction IConcept IEntity ILabel IProp IResource IStatement ITerritory IUser IValue).


In [344]:
# factory for making entity objects, contains defaults with "prerequisities"

class InkVisitorJSONObjectFactory:

    classes = json_classes

    json_class_defaults = {
        'IAction':{
            'class':'A', 'id':'','legacyId':'', 'label':'', 'language':'', 'detail':'','data':{'entities':{'a1':[],'a2':[],'s':[]},'valencies':{'a1':'','a2':'','s':''}}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IConcept':{
            'class':'C', 'id':'', 'legacyId':'','label':'', 'language':'', 'detail':'','data':{}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IValue':{
            'class':'V', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'4'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IProp':{
            'bundleEnd':False,'bundleStart':False, 'certainty':'1', 'children':[], 'elvl':'1',  'id':'', 'logic':'1', 'mood':[], 'moodvariant':'1', 'bundleOperator':'=', 'type': {'id':'','elvl':'1','logic':'1','partitivity':'1','virtuality':'1'},'value':{'id':'','elvl':'1', 'logic':'1', 'partitivity':'1', 'virtuality':'1'}
        },
        'IResource':{
             'class':'R', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'link':''}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IStatement':{
             'class':'S', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'actants':[], 'actions':[], 'references':[],'tags':[],'territory': {'id':'','order':0}, 'text':''}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'ITerritory':{
            'class':'T', 'id':'', 'legacyId':'','label':'', 'language':'', 'detail':'','data':{'parent':{ "id": "T0", "order": 0 }}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
    }

    def __init__(self):
        pass

    def make(self, entity_name, override_object=None):
        if override_object is None:
            override_object = {}
        object = type(self).json_class_defaults[entity_name]
        object.update(override_object)
        return type(self).classes[entity_name](deepcopy(object))

    def validate_defaults(self):
        for e in self.json_class_defaults:
            test = self.make(e, self.json_class_defaults[e])
            d.logger.info(f"Class {e} validated.")


IOF = InkVisitorJSONObjectFactory()
IOF.validate_defaults()

2022-03-08 23:13:02,247 INFO Class IAction validated.
2022-03-08 23:13:02,252 INFO Class IConcept validated.
2022-03-08 23:13:02,257 INFO Class IValue validated.
2022-03-08 23:13:02,261 INFO Class IProp validated.
2022-03-08 23:13:02,266 INFO Class IResource validated.
2022-03-08 23:13:02,273 INFO Class IStatement validated.
2022-03-08 23:13:02,279 INFO Class ITerritory validated.


# Load input datasets

In [399]:
# empty value unifier
def unify_empty_value(df: pd.DataFrame, empty_values=None, unified_empty_value =''):
    if empty_values is None:
        empty_values = ['NA', "#N/A", "#VALUE!"]
    for naner in empty_values:
        df = df.replace(naner,unified_empty_value)
    df. fillna(unified_empty_value, inplace=True)
    return df



# load all input tables
tables = {}
header_infos = {}
for key, sheet in input_sheets.items():
    logger.info(f"Calling for {key} with sheet_name {sheet[0]}.")
    tables[key], header_infos[key] = d.load_df_from_gsheet(sheet[0],root_sheet_url + sheet[1], sheet[0], fromCache=True, header_in_row=sheet[2], clean=True, fillna=True, cleanByColumn="label", parse_hyperlink_formulas=True)
    tables[key] = unify_empty_value(tables[key])
    header_infos[key] = unify_empty_value(header_infos[key])

    # code for legacyId copy and uuid creation
    tables[key]['legacyId'] = tables[key]['id'].copy()
    # inform instructive header about the new column and what to do with it
    header_infos[key]['legacyId'] = ""
    header_infos[key].at[3,'legacyId'] = "inside"
    tables[key]['id'] = tables[key].apply(lambda x: get_uuid_id(), axis=1)  # generate unique id for each row

# making empty table of values
tables['values'] = pd.DataFrame(columns=['id','value','origin'])

2022-03-09 07:59:33,034 INFO Calling for texts with sheet_name Texts.


20 2022-03-09 07:59:33 : Loading dataset Texts
20 2022-03-09 07:59:33 : Opting for variant header at row 5.
20 2022-03-09 07:59:39 : Hyperlinks were detected and transformed in columns ['edition_1', 'edition_2', 'edition_3', 'persons_index_link', 'places_index_link'].
20 2022-03-09 07:59:39 : Dropping empty columns in the dataset Texts : (1011, 113)


2022-03-09 07:59:39,297 INFO Calling for manuscripts with sheet_name Manuscripts.


20 2022-03-09 07:59:39 : Deleted 869 empty rows by label.
20 2022-03-09 07:59:39 : Loaded and prepared dataset Texts : (142, 113)
20 2022-03-09 07:59:39 : Making pickle cache of  Texts with separeted header file : (142, 113)
20 2022-03-09 07:59:39 : Loading dataset Manuscripts
20 2022-03-09 07:59:39 : Opting for variant header at row 5.


2022-03-09 07:59:40,989 INFO Calling for resources with sheet_name Resources.


20 2022-03-09 07:59:40 : Hyperlinks were detected and transformed in columns ['reproduction_online_url', 'reproduction_note'].
20 2022-03-09 07:59:40 : Dropping empty columns in the dataset Manuscripts : (999, 43)
20 2022-03-09 07:59:40 : Deleted 860 empty rows by label.
20 2022-03-09 07:59:40 : Loaded and prepared dataset Manuscripts : (139, 43)
20 2022-03-09 07:59:40 : Making pickle cache of  Manuscripts with separeted header file : (139, 43)
20 2022-03-09 07:59:41 : Loading dataset Resources
20 2022-03-09 07:59:41 : Opting for variant header at row 5.


2022-03-09 07:59:42,468 INFO Calling for actions with sheet_name Statements.


20 2022-03-09 07:59:42 : Hyperlinks were detected and transformed in columns [].
20 2022-03-09 07:59:42 : Dropping empty columns in the dataset Resources : (1000, 21)
20 2022-03-09 07:59:42 : Deleted 934 empty rows by label.
20 2022-03-09 07:59:42 : Loaded and prepared dataset Resources : (66, 21)
20 2022-03-09 07:59:42 : Making pickle cache of  Resources with separeted header file : (66, 21)
20 2022-03-09 07:59:42 : Loading dataset Statements
20 2022-03-09 07:59:42 : Opting for variant header at row 5.
20 2022-03-09 07:59:48 : Hyperlinks were detected and transformed in columns [].
20 2022-03-09 07:59:48 : Dropping empty columns in the dataset Statements : (1030, 74)
20 2022-03-09 07:59:48 : Deleted 588 empty rows by label.
20 2022-03-09 07:59:48 : Loaded and prepared dataset Statements : (442, 74)
20 2022-03-09 07:59:48 : Making pickle cache of  Statements with separeted header file : (442, 74)


2022-03-09 07:59:48,610 INFO Calling for concepts with sheet_name Concepts.


20 2022-03-09 07:59:49 : Loading dataset Concepts
20 2022-03-09 07:59:49 : Opting for variant header at row 5.
20 2022-03-09 07:59:53 : Hyperlinks were detected and transformed in columns [].
20 2022-03-09 07:59:53 : Dropping empty columns in the dataset Concepts : (3011, 60)
20 2022-03-09 07:59:53 : Deleted 634 empty rows by label.
20 2022-03-09 07:59:53 : Loaded and prepared dataset Concepts : (2377, 60)
20 2022-03-09 07:59:53 : Making pickle cache of  Concepts with separeted header file : (2377, 60)


In [386]:
header_infos['concepts']

,status,id,label,detail,language,wordnet_lemma_id,wordnet_synset_id,instance_entity_type,DEPRECATED_supertype0,DEPRECATED_supertype1,...,related_entity_label,related_entity_detail,masterlists_column_name,masterlists_column_value,text_example,note,recommendation,DEPRECATED_label_english,parsing_rows_explained,legacyId
0,,,,,,WordNet lemma (URI),Wordnet meaning (synset),This was for automatic instantiation of deprec...,,,...,,,,,"Discard, because the examples (incl. exact ref...",Save as one item of the multi-note.,Save as one item of the multi-note.,,Comment - sometimes important to read for pars...,
1,,,,,,,,,,,...,,,,,,,,,"""origin"" (for props).",
2,,,,,,,,,,,...,,,,,,note,note,,Prop type (for props) or further detail (for i...,
3,inside,inside,inside,inside,inside,inside?,inside?,discard,discard,discard,...,discard,discard,discard,discard,discard,inside,inside,discard,discard,inside


# Declaration of controlling classes

In [400]:
additional_entities = []

# for controlling entity and mapping of its fields
class EntityMapper:

    # simple inside values mapping from input_values in gsheets to inkVisitor enums
    # field: { FROM  : TO }
    enum_mapper = {'language': {"English":"eng","Latin":"lat","Occitan":"oci","Middle English":"enm"},"status":{"approved":"1","pending":"0","discouraged":"2","warning":"3"}}
    # status  Pending = "0",   Approved = "1",  Discouraged = "2",  Warning = "3",


    IOF = InkVisitorJSONObjectFactory()

    def __init__(self, entity_type, logger = d.logger):
        self.entity =  type(self).IOF.make(entity_type)
        self.logger = logger


    def get_entity_id(self,entity_string, origin = ""):
        id = ""
        # logger.info(f"Getting entity string {entity_string} in {origin}")
        try:
            if "C" in entity_string:
                id = tables['concepts'].loc[tables['concepts']['legacyId'] == entity_string,'id'].values[0]
            elif "V" in entity_string:
                ventity = self.make_ventity(entity_string, origin=origin)
                id = ventity['id']
            elif "M" in entity_string:
                id = tables['manuscripts'].loc[tables['manuscripts']['legacyId'] == entity_string,'id'].values[0]
            elif "A" in entity_string:
                id = tables['actions'].loc[tables['actions']['legacyId'] == entity_string,'id'].values[0]
            elif "R" in entity_string:
                id = tables['resources'].loc[tables['resources']['legacyId'] == entity_string,'id'].values[0]
            elif is_valid_uuid(entity_string):
                id = entity_string

        except IndexError as E:
            logger.info(f"Cannot get entity id from entity string {entity_string} in {origin}. {E}")

        if id != "" and isinstance(id, str):
            # logger.info(f"Got entity id {id} from entity string {entity_string} in {origin}")
            return id
        else:
            logger.error(f"Cannot get entity id from {entity_string}.")
            raise Exception(f"Cannot get entity id from {entity_string}.")


    def make_ventity(self, value_string, origin=""):
        # logger.info(f"Generating ventity from {value_string}.")
        # generate value entity object...
        ventity = IOF.make('IValue')
        ventity['id'] = get_uuid_id()
        ventity['label'] = value_string.replace("~V~","")

        # register ventity
        tables['values'] = tables['values'].append({'id':ventity['id'] ,'value':ventity['label'],"origin":""},ignore_index=True )
        additional_entities.append(ventity)

        # logger.info(f"Ventity id={ventity['id']} generated.")

        return ventity

    def make_rentity(self, label, url = "", origin=""):
        # logger.info(f"Generating rentity from {value_string}.")
        # generate resource entity object...
        rentity = IOF.make('IResource')
        rentity['id'] = get_uuid_id()
        rentity['label'] = label
        rentity['data']['link'] = url

        # register ventity
        tables['resources'] = tables['resources'].append({'id':rentity['id'] ,'value':rentity['label'],"origin":origin},ignore_index=True )
        additional_entities.append(rentity)

        # logger.info(f"Rentity id={rentity['id']} generated.")
        return rentity

    # interprets prop_type (should be always concept (?)) and input_value (should be concept or value string)
    # get ids of the prop_type and prop_value (possibly creates and register values object)
    # make iProp object
    # puts iProp object into the entity props property
    def hook_prop_object(self, prop_type, input_value, prop_source="",  origin=""):
        # allowed entities in type
        assert "C" in prop_type, f"Prop type unknown, C string entity expected? {prop_type}, {origin}"

        # allowed entities in input
        allowed_entities_classes = ['C','M','V']
        #hack for making ~V~string from input_values, which are not explicit
        if not any(c in input_value for c in allowed_entities_classes):
            input_value = "~V~"+input_value
        assert any(c in input_value for c in allowed_entities_classes), f"Prop value unknown, C string or V string entity expected? {input_value}, {origin}"

        prop_type_id = self.get_entity_id(prop_type, origin=origin)
        prop_value_id = self.get_entity_id(input_value, origin=origin)
        # logger.info(f"{prop_type_id}, {prop_value_id}")

        # make IProp object
        prop_object = IOF.make('IProp')
        prop_object['id'] = get_uuid_id()
        prop_object['type']['id'] = prop_type_id
        prop_object['value']['id'] = prop_value_id


        if prop_source !="": # means propvalue_2nd
            self.hook_2ndprop_into_props(prop_object,prop_source)
            pass
        else:
            # hook directly into the entity object
            self.hook_prop_into_props(prop_object)


    def hook_prop_into_props(self,prop_object):
        self.entity['props'].append(prop_object)

    def hook_2ndprop_into_props(self,prop_object,fst_prop_identification):  # identification by concept id
        key = 0
        keyId = self.get_entity_id(fst_prop_identification)
        assert len(keyId)>0, f"Cannot recognize entity id from {fst_prop_identification}"

        # count, value in enumerate(values)
        for count, po in enumerate(self.entity['props']):
            if po['type']['id'] == keyId and len(po['children']) == 0:  # I am counting on the fact, that if there relations from multiples, they are processed in the specific right order
               po['children'].append(prop_object)

    # method invoker for the INSIDE operation with concrete fields
    def update_inside_field(self, field_name, input_value, origin= ""):
        if input_value != '':

            if "#" in input_value or "~" in input_value:
                self.logger.info(f"ALERT # or ~ in the input value {input_value}")

            update_operation = "update_" + field_name
            update_func = getattr(self, update_operation, self.update_generic)
            update_func(field_name, input_value, origin)
        else:
            raise Exception(f"Trying to update {field_name} with empty input value.")

    #########################################################################################################
    # the naming of procedures corresponds to the name of the input_table fields,  used for inside operations

    def update_label_language(self, field_name, input_value, origin = ""):
        if input_value in type(self).enum_mapper['language']:
            self.entity['language'] = type(self).enum_mapper['language'][input_value]
        else:
            self.entity['language'] = input_value # will raise error

    def update_status(self, field_name, input_value, origin = ""):
        if input_value in type(self).enum_mapper['status']:
            self.entity['status'] = type(self).enum_mapper['status'][input_value]
        else:
            self.entity['status'] = input_value # will raise error

    def update_note(self, field_name, input_value, origin = ""):
        #self.logger.info(f"Updating note with {input_value}.")
        self.entity['notes'].append(input_value)

    def update_id(self, field_name, input_value, origin = ""):
        # self.entity['id'] = input_value
        self.entity['id'] = input_value

    def update_legacyId(self, field_name, input_value, origin = ""):
        # logger.info(f"Trying to set legacyId {type(input_value)}:'{input_value}' {origin}.")
        self.entity['legacyId'] = input_value

    def update_label(self, field_name, input_value, origin = ""):
        self.entity['label'] = "_I_"+input_value

    def update_generic(self, field_name, input_value, origin = ""):
        self.entity[field_name] = input_value



class TerritoryEntityMapper(EntityMapper):
    def __init__(self,entity_type, logger = d.logger):
        EntityMapper.__init__(self,entity_type, logger)

class ConceptEntityMapper(EntityMapper):
    def __init__(self,entity_type, logger = d.logger):
        EntityMapper.__init__(self,entity_type, logger)

class ActionEntityMapper(EntityMapper):
    def __init__(self,entity_type, logger = d.logger):
        EntityMapper.__init__(self,entity_type, logger)

class ResourceEntityMapper(EntityMapper):
    def __init__(self,entity_type, logger = d.logger):
        EntityMapper.__init__(self,entity_type, logger)

class ObjectEntityMapper(EntityMapper):
    def __init__(self,entity_type, logger = d.logger):
        EntityMapper.__init__(self,entity_type, logger)



class EntityMapperFactory:
    def __init__(self):
        pass

    def make(self,entity_name):
        if 'ITerritory' == entity_name:
            return TerritoryEntityMapper(entity_name)
        elif 'IAction' == entity_name:
            return ActionEntityMapper(entity_name)
        elif 'IConcept' == entity_name:
            return ConceptEntityMapper(entity_name)
        elif 'IResource' == entity_name:
            return ResourceEntityMapper(entity_name)
        elif 'IObject' == entity_name:
            return ObjectEntityMapper(entity_name)

        else:
            logger.warning(f"Unrecognized entity class in entity mapper. Is this right?")
            return EntityMapper(entity_name)




# CONTROL CLASS
class ParseController():
    def __init__(self, entity_list: [], keyword_row_id = 3,  logger = d.logger):
        self.entity_list = entity_list
        self.logger = logger
        self.parsers = {}
        self.js_objects = []

        for e in self.entity_list:
            if 'texts' in e:
                self.parsers[e] = TextParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            else:
                self.parsers[e] = Parser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)

    def load_json_objects(self):
        for name, p in self.parsers.items():
            self.js_objects = self.js_objects + p.js_objects


# WORKER CLASS
class Parser():
    EMP = EntityMapperFactory()

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        self.name = name
        self.logname = name.upper()
        self.input_header_df = header_df
        self.input_table_df = table_df
        self.prepared_table = pd.DataFrame()
        self.keyword_row_id =  keyword_row_id
        self.columns = self.input_header_df.columns.tolist()

        self.parsing_instruction = {}
        self.oper_columns = {'discard':[],'inside':[],'special':[],'unknown':[],'propvalue':[],'propvalue_2nd':[]}
        self.logger = logger

        # parsed json data holder
        self.js_objects = []

        # RUN
        self.process_header_instructions()
        self.prepare_input_table()

    # "parsing" instructions
    def process_header_instructions(self) -> (pd.DataFrame, pd.DataFrame):
        keyword_row = self.input_header_df.iloc[self.keyword_row_id]
        prop_type_row = self.input_header_df.iloc[self.keyword_row_id - 1]
        source_node_row = self.input_header_df.iloc[self.keyword_row_id - 2]

        log_uncertain_instructions = []

        for c in self.columns:
            instruction_candidate = str(keyword_row.at[c]).strip()
            prop_type_candidate = str(prop_type_row.at[c]).strip()
            source_node_candidate = str(source_node_row.at[c]).strip()

            if c == '':
                self.logger.error(f"{self.logname} There is empty column in the dataset.")
                raise Exception(f"{self.logname} There is empty column in the dataset.")

            if "?" in instruction_candidate or "?" in prop_type_candidate or "?" in source_node_candidate:
                log_uncertain_instructions.append(f"{c.upper()}:{instruction_candidate},{prop_type_candidate},{source_node_candidate}")
                instruction  = {'operation':'discard', 'target': None}
                self.oper_columns['discard'].append(c)

            if 'discard' in instruction_candidate:
                instruction  = {'operation':'discard', 'target': None}
                self.oper_columns['discard'].append(c)

            elif 'propvalue' ==  instruction_candidate:
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'propvalue', 'type': prop_type, 'source':source_node} # source can be ignored, because the iProp object is sitting inside of it
                    self.oper_columns['propvalue'].append(c)

            elif 'propvalue_2nd' in instruction_candidate:
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'propvalue_2nd', 'type': prop_type, 'source':source_node} # source can NOT be ignored, it signals which existing iProp object will hold this iProp object
                    self.oper_columns['propvalue_2nd'].append(c)

            elif 'special' in instruction_candidate:
                # looks for custom functions registered by column name
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                instruction  = {'operation':'special', 'type': prop_type, 'source':source_node}
                self.oper_columns['special'].append(c)

            elif 'inside' in instruction_candidate:
                if "?" in c:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'inside', 'target': None}
                    if len(prop_type_candidate) > 0:
                        instruction  = {'operation':'inside', 'target': prop_type_candidate}

                    self.oper_columns['inside'].append(c)

            else:
                instruction = {'operation':'unknown', 'target': None}
                self.oper_columns['unknown'].append(c)
            self.parsing_instruction[c] = instruction

        self.logger.info(f"{self.logname} Uncertain parsing instructions in {len(log_uncertain_instructions)} columns: " + " ".join(log_uncertain_instructions) + ".")
        return self.parsing_instruction

    def prepare_input_table(self):
        ip = self.input_table_df.copy()

        # discard  columns with discard and unknown operations
        ip.drop(columns=self.oper_columns['discard']+self.oper_columns['unknown'], inplace=True)

        self.logger.info(f"{self.logname} {len(self.oper_columns['discard']+self.oper_columns['unknown'])} columns have been dropped (discard:{len(self.oper_columns['discard'])}, unknown:{len(self.oper_columns['unknown'])}). Table now has {len(ip.columns)} columns, inside:{len(self.oper_columns['inside'])},propvalue:{len(self.oper_columns['propvalue'])}, special:{len(self.oper_columns['special'])}. Originally {self.input_table_df.shape[1]} columns.")

        self.prepared_table = ip


    def prepare_property(self):
        pass


    def make_row_object(self):
        class_name = table_to_entity[self.name]
        return type(self).EMP.make(class_name)

    def itemize_valuestring_for_multiples(self, value_with_multiples, origin="") -> []:
        values = []
        if isinstance(value_with_multiples,str):
            parsed_value = value_with_multiples.split('#')
            values =  [item.strip() for item in parsed_value]
        else:
            raise Exception(f"Expected value to be string. Got {type(value_with_multiples)}. {origin}")
        return values

    def parse_rows(self):
        for key, row in self.prepared_table.iterrows():
            # logger.info(f"{self.name} Processing row {key}")
            entity_mapper = self.make_row_object()
            for name, value in row.items():
                if name in self.parsing_instruction:
                    operation = self.parsing_instruction[name]
                else:
                    continue # silently ignore unknown columns
                #logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")

                # force string
                value = str(value)

                if operation['operation'] == 'inside' and value != '' and '?' not in name:
                    #logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    if operation['target']:
                        name = operation['target']
                    entity_mapper.update_inside_field(name,value,operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value))

                if operation['operation'] == 'propvalue' and value != '':
                    #logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    prop_type = operation['type']
                    if prop_type == '' or 'C' not in prop_type:
                        raise Exception(f"Propvalue does not have prop type defined. C entity-string expected, got {key}, {name}, {value}")
                    for item in self.itemize_valuestring_for_multiples(value):
                        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = item, origin = operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value))

                if operation['operation'] == 'propvalue_2nd' and value != '':
                    # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    prop_type = operation['type']
                    prop_source_name = operation['source']  # header_name,  we need concept_id

                    prop_source_id = self.parsing_instruction[prop_source_name]['type']
                    assert 'C' in prop_source_id, f"Trying to get to the concept of 1st level property to adress 2nd level property."

                    if prop_type == '' or 'C' not in prop_type:
                        raise Exception(f"Propvalue does not have prop type defined. C entity-string expected, got {key}, {name}, {value}")

                    origin = operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value)

                    for item in self.itemize_valuestring_for_multiples(value, origin=origin):
                        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = item, prop_source = prop_source_id, origin = origin)
                    pass

                if operation['operation'] == 'special' and value != "":
                    # logger.info(f"SPECIAL {operation} {value}")
                    func = getattr(self, 'special_'+name)
                    func(operation, value, entity_mapper)

            self.js_objects.append(entity_mapper.entity)
            # break


class TextParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    # special methods for fields, which needs fully individual processing
    def special_edition_1(self, operation, value, entity_mapper, field_name="special_edition1", ):
        # Parse this col. as "propvalue" - but you need to generate the target entities since they do not exist. How to do it: for any value, create an R entity with "label" = textual value in this col., "label language" = English, "status" = "approved", and "URL" = the hyperlink in the formula sitting on the textual value in this col. As usual, ignore NS and NA values (exact match) - do not import anything if the value is NA.
        # logger.info(f"Special edition1 running ...{operation} {value}")
        prop_type = operation['type']


        # make rentity
        # logger.info(f"Rentity making {value}")
        if "|" in value:
            data = value.split("|")
            label = data[0]
            url = data[1]
        else:
            url = ""
            label = value
            # logger.warning(f"Expected char | signaling url after label. Got just {value}")

        rentity = entity_mapper.make_rentity(label, url, origin=field_name)

        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = rentity['id'], origin = operation['operation'] +">"+ ":"+field_name + str(value))

    def special_edition_2(self, operation, value, entity_mapper):
        self.special_edition_1( operation, value, entity_mapper, field_name="edition_2")

    def special_edition_3(self, operation, value, entity_mapper):
        self.special_edition_1( operation, value, entity_mapper, field_name="edition_3")



In [348]:
# step 1: implement import of properties
# ... id  : with this David signal, that the value of the type in props object is  the highest entity itslef (e.g. text)
#     or other header_name for propvalue_2nd
# property type: C0732
# instruction propvalue
# inout table field: language_id
# value example : C0938


#intracacies,  multiples and value objects
#  BIG CONSEQUENCES : I need to create and fill the virtual dataset of value objects, which  will be appended to the entities.json



# Run

In [401]:
logger.info(f"Start")
#cp = ParseController(entity_list=['texts','manuscripts','resources','concepts','actions'])

cp = ParseController(entity_list=['texts','concepts'])
cp.parsers['texts'].parse_rows()
cp.parsers['concepts'].parse_rows()
logger.info(f"End")

2022-03-09 08:00:42,204 INFO Start
2022-03-09 08:00:42,211 INFO TEXTS Uncertain parsing instructions in 6 columns: TIMERELATION1_TARGET_ID:propvalue?,, TIMERELATION2_TYPE_CONCEPTIFIED_ID:proptype?,, TIMERELATION2_TARGET_ID:propvalue?,, TIMERELATION3_TYPE_CONCEPTIFIED_ID:proptype?,, TIMERELATION3_TARGET_ID:propvalue?,, TIMERELATION4_TARGET_ID:propvalue?,,.
2022-03-09 08:00:42,213 INFO TEXTS 80 columns have been dropped (discard:72, unknown:8). Table now has 40 columns, inside:9,propvalue:14, special:3. Originally 114 columns.
2022-03-09 08:00:42,218 INFO CONCEPTS Uncertain parsing instructions in 2 columns: WORDNET_LEMMA_ID:inside?,, WORDNET_SYNSET_ID:inside?,,.
2022-03-09 08:00:42,223 INFO CONCEPTS 45 columns have been dropped (discard:44, unknown:1). Table now has 16 columns, inside:10,propvalue:8, special:0. Originally 61 columns.
2022-03-09 08:02:02,715 INFO End


In [421]:
%reload_ext line_profiler

def profile():
    cp.parsers['concepts'].parse_rows()

%lprun -f EntityMapper.update_inside_field profile()

In [413]:
%lprun?

In [402]:
print(json.dumps(cp.parsers['texts'].js_objects[0]))

{"class": "T", "id": "0f243e16-6f2e-4441-8612-c5795cbf100d", "legacyId": "T1", "label": "_I_Process against Bernard Niort and his family", "language": "eng", "detail": "Early 1234.", "data": {"parent": {"id": "T0", "order": 0}}, "props": [{"bundleEnd": false, "bundleStart": false, "certainty": "1", "children": [], "elvl": "1", "id": "220fb083-0662-4da0-b20e-129af2255908", "logic": "1", "mood": [], "moodvariant": "1", "bundleOperator": "=", "type": {"id": "95e5725c-f5d9-4d92-96af-cdf88499e921", "elvl": "1", "logic": "1", "partitivity": "1", "virtuality": "1"}, "value": {"id": "ac814816-705e-41ae-b6b1-5e0e77f79eaf", "elvl": "1", "logic": "1", "partitivity": "1", "virtuality": "1"}}, {"bundleEnd": false, "bundleStart": false, "certainty": "1", "children": [], "elvl": "1", "id": "6709d257-4857-4247-b3eb-4366b8e2814e", "logic": "1", "mood": [], "moodvariant": "1", "bundleOperator": "=", "type": {"id": "8726db8b-9834-4300-a134-1c4bf309c816", "elvl": "1", "logic": "1", "partitivity": "1", "vi

In [403]:
logger.info(f"There is {len(additional_entities)} additionally created entities (e.g. values, resources ...).")

2022-03-09 08:07:31,069 INFO There is 219 additionally created entities (e.g. values, resources ...).


In [404]:
cp.load_json_objects()

# Output the parsed data

In [397]:
# save json object list in the text file
with open('../datasets/all-test/new_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(cp.js_objects, f)

# save json object list in the text file
with open('../datasets/all-test/additional_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(additional_entities, f)

# read  entities.json
with open('../datasets/all/entities.json','r') as f:
    #entities_content = f.readlines()
    entities_content = f.read().replace('\n', '')

# read  entities.json
with open('../datasets/all-test/new_entities.json','r') as f:
    #entities_content = f.readlines()
    new_entities_content = f.read().replace('\n', '')

additional_entities_string = json.dumps(additional_entities)

# write new
with open('../datasets/all-test/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content = entities_content[0:-1] +", " + additional_entities_string[1:-1]+ ", " + new_entities_content[1:]
    f.write(str(merge_content))